In [1]:
from nltk.corpus import brown
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import string
from rake_nltk import Rake
import json
import nltk
import numpy as np
from tqdm import tqdm
import pandas as pd
import os, json
import pandas.io.json as pd_json
from eda import *
from words_refine import *

[nltk_data] Error loading brown: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 61] Connection refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


In [2]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

#     results= {}
#     for idx in range(len(feature_vals)):
#         results[feature_vals[idx]]=score_vals[idx]
    results = []
    for idx in range(len(feature_vals)):
        results.append(feature_vals[idx])
    
    return results

def load_json1(filename):
    data_list = []
    lines = open(filename,'r').readlines()
    for i in range(len(lines)):
        line = json.loads(lines[i])
        
        # refine origins
        tmp_origin = line['origins'] if line['origins'] else None
        tmp_origin = tmp_origin if tmp_origin is None else tmp_origin[0]
        
        # refine ingredients
        tmp_ingredients = line['ingredients'] if line['ingredients'] else None
        tmp_ingredients = tmp_ingredients['ingredients'] if tmp_ingredients is not None else tmp_ingredients
        tmp_ingredients = tmp_ingredients if tmp_ingredients else None
        # merge to one sentences
        tmp_ingredients = ", ".join(tmp_ingredients) if tmp_ingredients is not None else None
        
        # refine allergens
        tmp_allergens =  line['allergens'] if line['allergens'] else None
        # merge to one sentences
        tmp_allergens = ", ".join(tmp_allergens) if tmp_allergens is not None else None
        
        tmp_description = line['description'] if line['description'] else None
        
        # 
        if line['breadcrumb'] is None:
            dic = {'name':line['name'], 'description': tmp_description,'origins':tmp_origin,
                   'allergens':tmp_allergens,'ingredients':tmp_ingredients,
                  'salePrice':line['price']['salePrice'],'savePrice':line['price']['savePrice'],
                  'label_1': None,
                  'label_2': None,
                  'label_3': None,
                  'tf-idf' : []}            
        else:
            dic = {'name':line['name'], 'description': tmp_description,'origins':tmp_origin,
                   'allergens':tmp_allergens,'ingredients':tmp_ingredients,
                  'salePrice':line['price']['salePrice'],'savePrice':line['price']['savePrice'],
                  'label_1':line['breadcrumb']['department']['name'],
                  'label_2':line['breadcrumb']['aisle']['name'],
                  'label_3': line['breadcrumb']['shelf']['name'],
                  'tf-idf' : []}
        data_list.append(dic)
    #json_data = json.dumps(data)
    df = pd.DataFrame.from_dict(data_list)
    return df

# load json with eda
def load_json2(filename):
    data_list = []
    lines = open(filename,'r').readlines()
    for i in range(len(lines)):
        line = json.loads(lines[i])
        
        # refine origins
        tmp_origin = line['origins'] if line['origins'] else None
        tmp_origin = tmp_origin if tmp_origin is None else tmp_origin[0]
        
        # refine ingredients
        tmp_ingredients = line['ingredients'] if line['ingredients'] else None
        tmp_ingredients = tmp_ingredients['ingredients'] if tmp_ingredients is not None else tmp_ingredients
        tmp_ingredients = tmp_ingredients if tmp_ingredients else None
        # merge to one sentences
        tmp_ingredients = ", ".join(tmp_ingredients) if tmp_ingredients is not None else None
        
        # refine allergens
        tmp_allergens =  line['allergens'] if line['allergens'] else None
        # merge to one sentences
        tmp_allergens = ", ".join(tmp_allergens) if tmp_allergens is not None else None
        
        # EDA for description and ingredient
        tmp_description = line['description'] if line['description'] else None
        alpha = 0.1
        #print (i)
        if tmp_description is not None:
            tmp_description = eda(tmp_description, alpha_sr=alpha, alpha_ri=alpha,
                                  alpha_rs=alpha, p_rd=alpha, num_aug=9)
            tmp_description = ". ".join(tmp_description)
        if tmp_ingredients is not None:
            tmp_ingredients = eda(tmp_ingredients, alpha_sr=alpha, alpha_ri=alpha,
                                  alpha_rs=alpha, p_rd=alpha, num_aug=9)
            tmp_ingredients = ". ".join(tmp_ingredients)
        
        # 
        if line['breadcrumb'] is None:
            dic = {'name':line['name'], 'description': tmp_description,'origins':tmp_origin,
                   'allergens':tmp_allergens,'ingredients':tmp_ingredients,
                  'salePrice':line['price']['salePrice'],'savePrice':line['price']['savePrice'],
                  'label_1': None,
                  'label_2': None,
                  'label_3': None,
                  'tf-idf' : []}            
        else:
            dic = {'name':line['name'], 'description': tmp_description,'origins':tmp_origin,
                   'allergens':tmp_allergens,'ingredients':tmp_ingredients,
                  'salePrice':line['price']['salePrice'],'savePrice':line['price']['savePrice'],
                  'label_1':line['breadcrumb']['department']['name'],
                  'label_2':line['breadcrumb']['aisle']['name'],
                  'label_3': line['breadcrumb']['shelf']['name'],
                  'tf-idf' : []}
        data_list.append(dic)
    #json_data = json.dumps(data)
    df = pd.DataFrame.from_dict(data_list)
    return df

def add_keyword(data):
    tmp = data["description"].dropna()
    cv=CountVectorizer(stop_words='english')
    word_count_vector=cv.fit_transform(tmp)
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    
    feature_names=cv.get_feature_names()
    for i in range(data.shape[0]):
        des = data.iloc[i]['description']
        if des is not None:
            tf_idf_vector=tfidf_transformer.transform(cv.transform([des]))
            sorted_items=sort_coo(tf_idf_vector.tocoo())
            keywords=extract_topn_from_vector(feature_names,sorted_items,5)
            data.at[i,'tf-idf'] = keywords
    
    return data

In [3]:
browse_df = load_json1('browse.json')
special_df = load_json1('specials.json')
# browse_df.to_csv('browse_df.csv', sep=',', header=True, index=True)
# special_df.to_csv('special_df.csv', sep=',', header=True, index=True)
br_eda_df = load_json2('browse.json')
sp_eda_df = load_json2('specials.json')

data = br_eda_df.append(sp_eda_df,ignore_index=True)
data = add_keyword(data)

In [10]:
# data.to_csv('dataset.csv', sep=',', header=True, index=True)

a = list(data['label_3'].unique())
len(a)

456

In [5]:
# tmp = data["description"].dropna()
# cv=CountVectorizer(stop_words='english')
# word_count_vector=cv.fit_transform(tmp)
# tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
# tfidf_transformer.fit(word_count_vector)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

# feature_names=cv.get_feature_names()
# tf_idf_vector=tfidf_transformer.transform(cv.transform([tmp[0]]))
# sorted_items=sort_coo(tf_idf_vector.tocoo())
# keywords=extract_topn_from_vector(feature_names,sorted_items,3)

In [6]:
# recipes = pd.read_csv('recipes.csv', sep=",", header=0)
# recipes.drop('url',axis = 1, inplace = True)